<a href="https://colab.research.google.com/github/wlvvs/FY22-Q2-AI-Hackathon-Anomaly-Detection/blob/master/notebooks/AnomalyDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import pandas as pd

Dentro del repositorio de Github que se estará usando, está el dataset base con el cual se da inicio al análisis de información

In [79]:
url ='https://raw.githubusercontent.com/wlvvs/FY22-Q2-AI-Hackathon-Anomaly-Detection/master/data/raw/ABCDPublishingStatistics2017-2022_Master.csv'

In [80]:
df = pd.read_csv(url,sep='|')

In [81]:
df.shape

(11719, 27)

El dataset consta de 27 filas y 11719 columnas y está separado por medio del símbolo pipe (|). Para realizar el análisis, se realizará una extracción de columnas con base en la utilidad al momento de definir un método de entrenamiento

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11719 entries, 0 to 11718
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   PKG_ID              11719 non-null  object
 1   PKG_JURIS           11719 non-null  object
 2   PKG_STATUS          11719 non-null  object
 3   PKG_TITLE           11719 non-null  object
 4   PKG_ISSUE_DATE      11719 non-null  object
 5   PKG_EFFECTIVE_DATE  11719 non-null  object
 6   PKG_APPROVAL_DATE   11719 non-null  object
 7   PKG_SUBMIT_DATE     11245 non-null  object
 8   PKG_RDY_DATE        11245 non-null  object
 9   PKG_FILED_DATE      11719 non-null  object
 10  PKG_PRINT_REQ_FLAG  2173 non-null   object
 11  PKG_DELETE_FLAG     11719 non-null  object
 12  PKG_FILING_NO       11719 non-null  object
 13  PKG_CREATE_DATE     11719 non-null  object
 14  PKG_ORG             11719 non-null  int64 
 15  PKG_RET_WHOL        11719 non-null  int64 
 16  PKG_PROJ_TYPE       11

In [83]:
df = df[['PKG_ID', 'PKG_JURIS', 'PKG_ISSUE_DATE', 'PKG_EFFECTIVE_DATE', 'PKG_APPROVAL_DATE', 'PKG_FILED_DATE','PKG_CREATE_DATE', 'PKG_ORG', 'PKG_RET_WHOL', 'PKG_PROJ_TYPE', 'PKG_FILING_TYPE', 'PKG_BUS_UNIT', 'PKG_ADVICE_LETTER', 'PKG_PUBLISHED_DATE', 'PKG_AUTO_PROCESS']]

In [84]:
df.shape

(11719, 15)

In [85]:
df.isna().sum()

PKG_ID                 0
PKG_JURIS              0
PKG_ISSUE_DATE         0
PKG_EFFECTIVE_DATE     0
PKG_APPROVAL_DATE      0
PKG_FILED_DATE         0
PKG_CREATE_DATE        0
PKG_ORG                0
PKG_RET_WHOL           0
PKG_PROJ_TYPE          0
PKG_FILING_TYPE        0
PKG_BUS_UNIT           0
PKG_ADVICE_LETTER      0
PKG_PUBLISHED_DATE    50
PKG_AUTO_PROCESS       0
dtype: int64

In [91]:
df = df.dropna()

In [92]:
df.isna().sum()

PKG_ID                0
PKG_JURIS             0
PKG_ISSUE_DATE        0
PKG_EFFECTIVE_DATE    0
PKG_APPROVAL_DATE     0
PKG_FILED_DATE        0
PKG_CREATE_DATE       0
PKG_ORG               0
PKG_RET_WHOL          0
PKG_PROJ_TYPE         0
PKG_FILING_TYPE       0
PKG_BUS_UNIT          0
PKG_ADVICE_LETTER     0
PKG_PUBLISHED_DATE    0
PKG_AUTO_PROCESS      0
dtype: int64

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11669 entries, 0 to 11718
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   PKG_ID              11669 non-null  object
 1   PKG_JURIS           11669 non-null  object
 2   PKG_ISSUE_DATE      11669 non-null  object
 3   PKG_EFFECTIVE_DATE  11669 non-null  object
 4   PKG_APPROVAL_DATE   11669 non-null  object
 5   PKG_FILED_DATE      11669 non-null  object
 6   PKG_CREATE_DATE     11669 non-null  object
 7   PKG_ORG             11669 non-null  int64 
 8   PKG_RET_WHOL        11669 non-null  int64 
 9   PKG_PROJ_TYPE       11669 non-null  int64 
 10  PKG_FILING_TYPE     11669 non-null  int64 
 11  PKG_BUS_UNIT        11669 non-null  int64 
 12  PKG_ADVICE_LETTER   11669 non-null  int64 
 13  PKG_PUBLISHED_DATE  11669 non-null  object
 14  PKG_AUTO_PROCESS    11669 non-null  object
dtypes: int64(6), object(9)
memory usage: 1.4+ MB


In [94]:
df.sample(3)

,PKG_ID,PKG_JURIS,PKG_ISSUE_DATE,PKG_EFFECTIVE_DATE,PKG_APPROVAL_DATE,PKG_FILED_DATE,PKG_CREATE_DATE,PKG_ORG,PKG_RET_WHOL,PKG_PROJ_TYPE,PKG_FILING_TYPE,PKG_BUS_UNIT,PKG_ADVICE_LETTER,PKG_PUBLISHED_DATE,PKG_AUTO_PROCESS
8405,FC-20-0004,FCC,16/06/2020 00:00,01/07/2020 00:00,01/07/2020 06:30,16/06/2020 10:45,09/06/2020 10:21,8,0,2,5,34,0,01/07/2020 08:23,N
2762,FC-17-0036,FCC,15/12/2017 00:00,01/01/2018 00:00,01/01/2018 07:30,14/12/2017 16:44,06/12/2017 09:42,2,0,0,0,0,0,02/01/2018 11:19,N
981,SC-17-0040,SOUTH CAROLINA,10/04/2017 00:00,10/04/2017 00:00,10/04/2017 06:30,30/03/2017 09:15,28/03/2017 08:56,1,1,12,2,1,0,10/04/2017 07:28,Z


La mayoria de los campos vienen expresados como objetos, lo cual puede significar un problema al momento de trabajar con dicha información, por lo que se tienen que transformar en un tipo de dato establecido que permita la gestión de la información.
Comenzaremos por cambiar la definición de objeto y colocarla como datos de tipo fecha o cadenas de caracteres según sea el caso necesario

In [95]:
df_dates= df[['PKG_ISSUE_DATE', 'PKG_EFFECTIVE_DATE', 'PKG_APPROVAL_DATE', 'PKG_FILED_DATE','PKG_CREATE_DATE', 'PKG_PUBLISHED_DATE']]

In [96]:
import dateutil.parser

all_true = True
for row in df_dates.values:
    flag = True
    print(row)
    for i in range(len(row[:-1])):
        
        try:
            parsed1 = dateutil.parser.parse(row[i],  dayfirst=True)
            parsed2 = dateutil.parser.parse(row[i+1], dayfirst=True)
            
            print(parsed1, type(parsed1))
            print(parsed2, type(parsed2))

            flag = flag and (parsed1 <= parsed2)
        except Exception as e:
            flag = True

        if not flag:
            break
    
    all_true = all_true and flag
    if not all_true:
        print(row)
        # break

    break

print(all_true)

['01/01/2017 00:00' '01/01/2017 00:00' '01/01/2017 06:30'
 '17/11/2016 08:36' '23/09/2016 14:41' '01/01/2017 07:25']
2017-01-01 00:00:00 <class 'datetime.datetime'>
2017-01-01 00:00:00 <class 'datetime.datetime'>
2017-01-01 00:00:00 <class 'datetime.datetime'>
2017-01-01 06:30:00 <class 'datetime.datetime'>
2017-01-01 06:30:00 <class 'datetime.datetime'>
2016-11-17 08:36:00 <class 'datetime.datetime'>
['01/01/2017 00:00' '01/01/2017 00:00' '01/01/2017 06:30'
 '17/11/2016 08:36' '23/09/2016 14:41' '01/01/2017 07:25']
False


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11669 entries, 0 to 11718
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   PKG_ID              11669 non-null  object
 1   PKG_JURIS           11669 non-null  object
 2   PKG_ISSUE_DATE      11669 non-null  object
 3   PKG_EFFECTIVE_DATE  11669 non-null  object
 4   PKG_APPROVAL_DATE   11669 non-null  object
 5   PKG_FILED_DATE      11669 non-null  object
 6   PKG_CREATE_DATE     11669 non-null  object
 7   PKG_ORG             11669 non-null  int64 
 8   PKG_RET_WHOL        11669 non-null  int64 
 9   PKG_PROJ_TYPE       11669 non-null  int64 
 10  PKG_FILING_TYPE     11669 non-null  int64 
 11  PKG_BUS_UNIT        11669 non-null  int64 
 12  PKG_ADVICE_LETTER   11669 non-null  int64 
 13  PKG_PUBLISHED_DATE  11669 non-null  object
 14  PKG_AUTO_PROCESS    11669 non-null  object
dtypes: int64(6), object(9)
memory usage: 1.4+ MB


In [100]:
df.columns

Index(['PKG_ID', 'PKG_JURIS', 'PKG_ISSUE_DATE', 'PKG_EFFECTIVE_DATE',
       'PKG_APPROVAL_DATE', 'PKG_FILED_DATE', 'PKG_CREATE_DATE', 'PKG_ORG',
       'PKG_RET_WHOL', 'PKG_PROJ_TYPE', 'PKG_FILING_TYPE', 'PKG_BUS_UNIT',
       'PKG_ADVICE_LETTER', 'PKG_PUBLISHED_DATE', 'PKG_AUTO_PROCESS'],
      dtype='object')

In [99]:
df.values

array([['AL-17-0002', 'ALABAMA', '01/01/2017 00:00', ..., 0,
        '01/01/2017 07:25', 'Z'],
       ['LA-17-0001', 'LOUISIANA', '01/01/2017 00:00', ..., 0,
        '01/01/2017 07:28', 'Z'],
       ['LA-17-0002', 'LOUISIANA', '01/01/2017 00:00', ..., 0,
        '01/01/2017 07:28', 'Z'],
       ...,
       ['TX-22-0013', 'TEXAS', '15/02/2022 00:00', ..., 0,
        '15/02/2022 11:35', 'N'],
       ['WG-22-0008', 'WISCONSIN', '15/02/2022 00:00', ..., 0,
        '15/02/2022 07:02', 'N'],
       ['SC-22-0008', 'SOUTH CAROLINA', '15/02/2022 00:00', ..., 0,
        '15/02/2022 07:05', 'Z']], dtype=object)

Revisa el caso que investigaste en liga y replantea tu modelo
https://www.statology.org/logistic-regression-python/